In [1]:
#Site CES 2025
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Configurer Selenium avec WebDriver Manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Charger la page principale
url = "https://exhibitors.ces.tech/8_0/#/searchtype/country/search/France/show/all"
driver.get(url)

# Attendre que la première liste de cartes charge
WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.js-Card'))
)

# Affichez un message pour signaler que vous pouvez scroller
print("Défilez manuellement dans le navigateur pour charger toutes les entreprises.")
print("Scrollez jusqu'à ce que plus aucune entreprise ne se charge, puis revenez ici.")
input("Appuyez sur Entrée lorsque vous avez terminé le défilement.")

# Extraire les informations des cartes après défilement manuel
cards = driver.find_elements(By.CSS_SELECTOR, 'li.js-Card')
print(f"Total d'entreprises affichées : {len(cards)}")

# Liste pour stocker les entreprises
exposants = []

for index, card in enumerate(cards):
    try:
        # Récupérer le titre
        try:
            title_element = card.find_element(By.CSS_SELECTOR, 'h3.card-Title')
            name = title_element.text
        except:
            title_element = card.find_element(By.CSS_SELECTOR, 'a')
            name = title_element.text

        # Récupérer le lien
        try:
            link_element = card.find_element(By.CSS_SELECTOR, 'a')  # Modifier ici si nécessaire
            link = link_element.get_attribute('href')
        except Exception as e:
            link = None
            print(f"Erreur lors de l'extraction du lien pour {name}: {e}")

        exposants.append({'Nom': name, 'Lien': link})

    except Exception as e:
        print(f"Erreur inattendue pour la carte {index + 1} : {e}")
        continue

# Vérifiez les entreprises sans lien
print(f"Entreprises sans lien : {[e for e in exposants if e['Lien'] is None]}")

# Extraire les détails pour chaque exposant
for exposant in exposants:
    try:
        if exposant['Lien']:
            # Naviguer vers la page entreprise
            driver.get(exposant['Lien'])

            # Attendre que la page charge les détails
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'h2.color-brand-02'))
            )

            # Description
            try:
                description = driver.find_element(By.CSS_SELECTOR, 'p.js-read-more.animated').text
                exposant['Description'] = description if description else "Non disponible"
            except Exception as e:
                exposant['Description'] = "Non disponible"
                print(f"Erreur lors de l'extraction de la description pour {exposant['Nom']}: {e}")

            # Catégories de produits
            try:
                categories = driver.find_elements(By.CSS_SELECTOR, 'div.grid.grid-3-col.grid__centered a')
                exposant['Catégories'] = [cat.text for cat in categories] if categories else []
            except Exception as e:
                exposant['Catégories'] = []
                print(f"Erreur lors de l'extraction des catégories pour {exposant['Nom']}: {e}")

            # Site web
            try:
                website = driver.find_element(By.CSS_SELECTOR, 'ul.showcase-web-phone a').get_attribute('href')
                exposant['Website'] = website if website else "Non disponible"
            except Exception as e:
                exposant['Website'] = "Non disponible"
                print(f"Erreur lors de l'extraction du site web pour {exposant['Nom']}: {e}")

            # LinkedIn
            try:
                linkedin = driver.find_element(By.CSS_SELECTOR, 'div.showcase-social.center.tc a').get_attribute('href')
                exposant['LinkedIn'] = linkedin if linkedin else "Non disponible"
            except Exception as e:
                exposant['LinkedIn'] = "Non disponible"
                print(f"Erreur lors de l'extraction du LinkedIn pour {exposant['Nom']}: {e}")

            # Adresse
            try:
                address = driver.find_element(By.CSS_SELECTOR, 'p.showcase-address.tc').text
                exposant['Adresse'] = address if address else "Non disponible"
            except Exception as e:
                exposant['Adresse'] = "Non disponible"
                print(f"Erreur lors de l'extraction de l'adresse pour {exposant['Nom']}: {e}")
        else:
            print(f"Pas de lien trouvé pour {exposant['Nom']}.")

    except Exception as e:
        print(f"Erreur lors de l'extraction des détails pour {exposant['Nom']} : {e}")

# Fermer le navigateur
driver.quit()

# Afficher les résultats
print(f"Nombre d'entreprises extraites : {len(exposants)}")
for exposant in exposants[:10]:  # Afficher les 10 premières entreprises pour vérification
    print(exposant)


In [ ]:
exposants

In [ ]:
import csv
# Exporter les données en CSV
csv_filename = "exposants.csv"
csv_columns = ['Nom', 'Lien', 'Description', 'Catégories', 'Website', 'LinkedIn', 'Adresse']

try:
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for exposant in exposants:
            # Convertir la liste des catégories en chaîne séparée par des virgules
            exposant['Catégories'] = ", ".join(exposant.get('Catégories', []))
            writer.writerow(exposant)
    print(f"Les données ont été exportées avec succès dans le fichier '{csv_filename}'.")
except IOError as e:
    print(f"Erreur lors de l'exportation en CSV : {e}")

In [ ]:
#Site CES 2025 - AI World
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Configurer Selenium avec WebDriver Manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Charger la page principale
url = "https://exhibitors.ces.tech/8_0/#/searchtype/category/search/191/show/all"
driver.get(url)

# Attendre que la première liste de cartes charge
WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.js-Card'))
)

# Affichez un message pour signaler que vous pouvez scroller
print("Défilez manuellement dans le navigateur pour charger toutes les entreprises.")
print("Scrollez jusqu'à ce que plus aucune entreprise ne se charge, puis revenez ici.")
input("Appuyez sur Entrée lorsque vous avez terminé le défilement.")

# Extraire les informations des cartes après défilement manuel
cards = driver.find_elements(By.CSS_SELECTOR, 'li.js-Card')
print(f"Total d'entreprises affichées : {len(cards)}")

# Liste pour stocker les entreprises
exposants = []

for index, card in enumerate(cards):
    try:
        # Récupérer le titre
        try:
            title_element = card.find_element(By.CSS_SELECTOR, 'h3.card-Title')
            name = title_element.text
        except:
            title_element = card.find_element(By.CSS_SELECTOR, 'a')
            name = title_element.text

        # Récupérer le lien
        try:
            link_element = card.find_element(By.CSS_SELECTOR, 'a')  # Modifier ici si nécessaire
            link = link_element.get_attribute('href')
        except Exception as e:
            link = None
            print(f"Erreur lors de l'extraction du lien pour {name}: {e}")

        exposants.append({'Nom': name, 'Lien': link})

    except Exception as e:
        print(f"Erreur inattendue pour la carte {index + 1} : {e}")
        continue

# Vérifiez les entreprises sans lien
print(f"Entreprises sans lien : {[e for e in exposants if e['Lien'] is None]}")

# Extraire les détails pour chaque exposant
for exposant in exposants:
    try:
        if exposant['Lien']:
            # Naviguer vers la page entreprise
            driver.get(exposant['Lien'])

            # Attendre que la page charge les détails
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'h2.color-brand-02'))
            )

            # Description
            try:
                description = driver.find_element(By.CSS_SELECTOR, 'p.js-read-more.animated').text
                exposant['Description'] = description if description else "Non disponible"
            except Exception as e:
                exposant['Description'] = "Non disponible"
                print(f"Erreur lors de l'extraction de la description pour {exposant['Nom']}: {e}")

            # Catégories de produits
            try:
                categories = driver.find_elements(By.CSS_SELECTOR, 'div.grid.grid-3-col.grid__centered a')
                exposant['Catégories'] = [cat.text for cat in categories] if categories else []
            except Exception as e:
                exposant['Catégories'] = []
                print(f"Erreur lors de l'extraction des catégories pour {exposant['Nom']}: {e}")

            # Site web
            try:
                website = driver.find_element(By.CSS_SELECTOR, 'ul.showcase-web-phone a').get_attribute('href')
                exposant['Website'] = website if website else "Non disponible"
            except Exception as e:
                exposant['Website'] = "Non disponible"
                print(f"Erreur lors de l'extraction du site web pour {exposant['Nom']}: {e}")

            # LinkedIn
            try:
                linkedin = driver.find_element(By.CSS_SELECTOR, 'div.showcase-social.center.tc a').get_attribute('href')
                exposant['LinkedIn'] = linkedin if linkedin else "Non disponible"
            except Exception as e:
                exposant['LinkedIn'] = "Non disponible"
                print(f"Erreur lors de l'extraction du LinkedIn pour {exposant['Nom']}: {e}")

            # Adresse
            try:
                address = driver.find_element(By.CSS_SELECTOR, 'p.showcase-address.tc').text
                exposant['Adresse'] = address if address else "Non disponible"
            except Exception as e:
                exposant['Adresse'] = "Non disponible"
                print(f"Erreur lors de l'extraction de l'adresse pour {exposant['Nom']}: {e}")
        else:
            print(f"Pas de lien trouvé pour {exposant['Nom']}.")

    except Exception as e:
        print(f"Erreur lors de l'extraction des détails pour {exposant['Nom']} : {e}")

# Fermer le navigateur
driver.quit()

# Afficher les résultats
print(f"Nombre d'entreprises extraites : {len(exposants)}")
for exposant in exposants[:10]:  # Afficher les 10 premières entreprises pour vérification
    print(exposant)


In [ ]:
import csv
# Exporter les données en CSV
csv_filename = "exposants.csv"
csv_columns = ['Nom', 'Lien', 'Description', 'Catégories', 'Website', 'LinkedIn', 'Adresse']

try:
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for exposant in exposants:
            # Convertir la liste des catégories en chaîne séparée par des virgules
            exposant['Catégories'] = ", ".join(exposant.get('Catégories', []))
            writer.writerow(exposant)
    print(f"Les données ont été exportées avec succès dans le fichier '{csv_filename}'.")
except IOError as e:
    print(f"Erreur lors de l'exportation en CSV : {e}")

In [ ]:
#Les pepites tech
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time

# Configurer Selenium avec WebDriver Manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Charger la page principale
url = "https://lespepitestech.com/best-of-pepites"
driver.get(url)

# Attendre que la première page charge
WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.s-e-content'))
)

# Afficher un message pour naviguer manuellement à la bonne page de départ
input("Naviguez manuellement vers la page de départ, puis appuyez sur Entrée pour commencer le scraping des pages suivantes.")

# Liste pour stocker les entreprises
entreprises = []

# Fonction pour scraper les données d'une page
def scraper_page():
    cards = driver.find_elements(By.CSS_SELECTOR, 'div.s-e-content')
    for card in cards:
        try:
            entreprise = {}

            # Nom de l'entreprise
            try:
                nom = card.find_element(By.CSS_SELECTOR, 'div.s-e-title h3').text
                entreprise['Nom'] = nom
            except Exception:
                entreprise['Nom'] = "Non disponible"

            # Description de l'entreprise
            try:
                description = card.find_element(By.CSS_SELECTOR, 'div.s-u-summary').text
                entreprise['Description'] = description
            except Exception:
                entreprise['Description'] = "Non disponible"

            # Catégories
            try:
                categories = card.find_elements(By.CSS_SELECTOR, 'div.lpt-dropdown-category span.tag a')
                entreprise['Catégories'] = [cat.text for cat in categories]
            except Exception:
                entreprise['Catégories'] = []

            # Votes
            try:
                votes = card.find_element(By.CSS_SELECTOR, 'div.counter.card-vote .alternate-votes-display').text.strip()
                entreprise['Votes'] = votes
            except Exception:
                entreprise['Votes'] = "Non disponible"

            # Commentaires
            try:
                commentaires = card.find_element(By.CSS_SELECTOR, 'div.counter.card-comment').text.strip()
                entreprise['Commentaires'] = commentaires
            except Exception:
                entreprise['Commentaires'] = "Non disponible"

            # Site Web
            try:
                site_web = card.find_element(By.CSS_SELECTOR, 'div.s-e-link a').get_attribute('href')
                entreprise['Site Web'] = site_web
            except Exception:
                entreprise['Site Web'] = "Non disponible"

            # Localisation
            try:
                localisation = card.find_element(By.CSS_SELECTOR, 'div.card-location a').get_attribute('data-original-title')
                entreprise['Localisation'] = localisation
            except Exception:
                entreprise['Localisation'] = "Non disponible"

            entreprises.append(entreprise)

        except Exception as e:
            print(f"Erreur lors de l'extraction des données pour une entreprise : {e}")
            continue

# Scraper les données des pages automatiquement
while True:
    scraper_page()
    try:
        # Trouver le bouton "Page suivante" via le titre ou le texte
        next_button = driver.find_element(By.CSS_SELECTOR, 'a[title="Aller à la page suivante"]')
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(3)  # Attendre que la nouvelle page charge
    except Exception:
        print("Aucune autre page trouvée. Fin du scraping.")
        break

# Fermer le navigateur
driver.quit()

# Exporter les données en CSV
csv_filename = "pepites_scraped.csv"
csv_columns = ['Nom', 'Description', 'Catégories', 'Votes', 'Commentaires', 'Site Web', 'Localisation']

try:
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for entreprise in entreprises:
            writer.writerow(entreprise)
    print(f"Les données ont été exportées avec succès dans le fichier '{csv_filename}'.")
except IOError as e:
    print(f"Erreur lors de l'exportation en CSV : {e}")

# Afficher les premières entreprises
for entreprise in entreprises[:10]:
    print(entreprise)
